<a href="https://colab.research.google.com/github/solharsh/Kaggle_Competitions/blob/master/Kaggle_Titanic_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3

In 1912, the British passenger ship Titanic struck an iceberg, which led to the death of over 1,500 passengers and crew, more than half the people on board.

You want to construct a model with Logistic Regression that uses available data about the passengers to predict their survival.

Use this dataset: titanic_dataset.csv (more information about this dataset is available on Kaggle (Links to an external site.)--however, ignore the other datasets there)

First, drop columns from the dataset that are obviously unnecessary for your model.
Then, assign the dataset's median Age to rows where the age data is missing.
Then, split the dataset into a training set and test set.
Fit your initial model, and identify and remove from consideration predictors that are not significant.
Plot a ROC curve and find the optimal cutoff probability
Update your predictions using the optimal cutoff
Finally, create a confusion matrix using your final predictions
 

Document your work and explain your decision making as you build your model.

Question:
1. First, drop columns from the dataset that are obviously unnecessary for your model.
2. Then, assign the dataset's median Age to rows where the age data is missing.
3. Then, split the dataset into a training set and test set.
4. Fit your initial model, and identify and remove from consideration predictors that are not significant.
5. Plot a ROC curve and find the optimal cutoff probability
6. Update your predictions using the optimal cutoff
7. Finally, create a confusion matrix using your final predictions



Strategising the Question:

Let's divide the process in 3 Parts :

A. Data Preparation

B. Model Fitting

C. Improving Accuracy






importing all important libraries for Statsmodels and Scikit-Learn:


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [0]:
train_df = pd.read_csv("/content/drive/My Drive/UC Machine Learning/Datasets/titanic_dataset.csv") #Reading the CSV file and creating dataframe

In [0]:
pred_df = pd.read_csv("/content/drive/My Drive/UC Machine Learning/Datasets/test.csv")

In [0]:
# append the two datasets for feature engineering
train_df["dataset"] = "train"
pred_df["dataset"] = "pred"
data_df = train_df.append(pred_df, sort=True)

From a sample of the RMS Titanic data, we can see the various features present for each passenger on the ship:

- Survived: Outcome of survival (0 = No; 1 = Yes)
- Pclass: Socio-economic class (1 = Upper class; 2 = Middle class; 3 = Lower class)
- Name: Name of passenger
- Sex: Sex of the passenger
- Age: Age of the passenger (Some entries contain NaN)
- SibSp: Number of siblings and spouses of the passenger aboard
- Parch: Number of parents and children of the passenger aboard
- Ticket: Ticket number of the passenger
- Fare: Fare paid by the passenger
- Cabin Cabin number of the passenger (Some entries contain NaN)
- Embarked: Port of embarkation of the passenger (C = Cherbourg; Q = Queenstown; S = Southampton)

In [0]:
# show missing values
data_df.isnull().sum()

Age             263
Cabin          1014
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
dataset           0
dtype: int64

The 'Name' column contains interesting information: the family name and the title. The title can give us relevant information about the marital status and age. The family name can help us to find if some persons of the individual's family survived.


In [0]:
# clean name and extracting Title

data_df['Title'] = data_df['Name'].str.extract('([A-Za-z]+)\.', expand=True)

# replace rare titles with more common ones
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr', 'Don': 'Mr', 'Mme': 'Miss',
          'Jonkheer': 'Mr', 'Lady': 'Mrs', 'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}

data_df.replace({'Title': mapping}, inplace=True)

A lot of age values are missing. We can impute the using the median of the persons with the same title.

In [0]:
titles = list(data_df.Title.value_counts().index)

# for each title, impute missing age by the median of the persons with the same title
for title in titles:
    age_to_impute = data_df.groupby('Title')['Age'].median()[title]
    data_df.loc[(data_df['Age'].isnull()) & (data_df['Title'] == title), 'Age'] = age_to_impute

The Parch variable corresponds to the number of parents or children aboard the Titanic. The SibSp variable corresponds to the number of siblings or spouses aboard the Titanic. We can add those two variables in order to get the size of the family that was onboard.

In [0]:
# compute family size
data_df['Family_Size'] = data_df['Parch'] + data_df['SibSp']

Now we can try to retrieve information about family survival. There are two ways to find people from the same family:

they have the same last name and they paid the same family fare
they have the same ticket id

In [0]:
# get the last name (family name): the string part before the ,
data_df['Last_Name'] = data_df['Name'].apply(lambda x: str.split(x, ",")[0])

# remove null fare values
data_df['Fare'].fillna(data_df['Fare'].mean(), inplace=True)

In [0]:
# get information about family survival using Last_Name and Fare

# 0.5: default value for no information
# 1: someone of the same family survived
# 0: we don't know if somebody survived but we know that somebody died

default_survival_value = 0.5
data_df['Family_Survival'] = default_survival_value

for grp, grp_df in data_df.groupby(['Last_Name', 'Fare']):
    # if a family group is found
    if (len(grp_df) != 1):
        # for every person, look for the other people from the same family
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            
            if (smax == 1.0): # if anyone in the family survived, assign 1
                data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin==0.0): # else if we saw someone dead, assign 0
                data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 0

print("Number of passengers with family survival information:", 
      data_df.loc[data_df['Family_Survival']!=0.5].shape[0])

Number of passengers with family survival information: 420


In [0]:
# get information about family survival using the Ticket number

for _, grp_df in data_df.groupby('Ticket'):
    # if a family group is found
    if (len(grp_df) != 1):
        # for every person, look for the other people from the same family
        for ind, row in grp_df.iterrows():
            if (row['Family_Survival'] == 0) | (row['Family_Survival']== 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0): # if anyone in the family survived, assign 1
                    data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (smin==0.0): # else if we saw someone dead, assign 0
                    data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 0
                        
print("Number of passenger with family survival information: " 
      +str(data_df[data_df['Family_Survival']!=0.5].shape[0]))

Number of passenger with family survival information: 546


Next we regroup Fare and Age into bins:

In [0]:
# fare bins
data_df['FareBin'] = pd.qcut(data_df['Fare'], 5)
label = LabelEncoder()
data_df['FareBin_Code'] = label.fit_transform(data_df['FareBin'])
data_df.drop(['Fare'], 1, inplace=True)

In [0]:
# age bins
data_df['AgeBin'] = pd.qcut(data_df['Age'], 4)
label = LabelEncoder()
data_df['AgeBin_Code'] = label.fit_transform(data_df['AgeBin'])
data_df.drop(['Age'], 1, inplace=True)

In [0]:
# encode sex variable
data_df['Sex'].replace(['male','female'],[0,1],inplace=True)

In [0]:
# choose features and labels
label = "Survived"
features = ["Pclass", "Sex", "Family_Size", "Family_Survival", "FareBin_Code", "AgeBin_Code"]

# split back data_df into train and prediction sets
train_df = data_df[data_df["dataset"] == "train"][features + [label]]
pred_df = data_df[data_df["dataset"] == "pred"][features]

# convert Survived variable to int for train dataset
train_df["Survived"] = train_df["Survived"].astype(np.int64)

In [0]:
train_df.head()

,Pclass,Sex,Family_Size,Family_Survival,FareBin_Code,AgeBin_Code,Survived
0,3,0,1,0.5,0,0,0
1,1,1,1,0.5,4,3,1
2,3,1,0,0.5,1,1,1
3,1,1,1,0.0,4,2,1
4,3,0,0,0.5,1,2,0


# Train and test KNN classifiers

In [0]:
# setup dataframes
X = train_df[features]
y = train_df['Survived']
X_pred = pred_df

# scale data for KNN classifier
std_scaler = StandardScaler()
X = std_scaler.fit_transform(X)
X_pred = std_scaler.transform(X_pred)

In [0]:
# setup parameters values for grid search
n_neighbors = [6,7,8,9,10,11,12,14,16,18,20,22]
weights = ['uniform', 'distance']
leaf_size = list(range(1,50,5))
hyperparams = {'weights': weights, 'leaf_size': leaf_size, 'n_neighbors': n_neighbors}


gd=GridSearchCV(estimator = KNeighborsClassifier(), param_grid = hyperparams, verbose=True, 
                cv=10, scoring = "roc_auc")
gd.fit(X, y)
print(gd.best_score_)
print(gd.best_estimator_)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8783125088419206
KNeighborsClassifier(algorithm='auto', leaf_size=26, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=18, p=2,
                     weights='uniform')


[Parallel(n_jobs=1)]: Done 2400 out of 2400 | elapsed:   12.1s finished


# Submit predictions

In [0]:
# make predictions
gd.best_estimator_.fit(X, y)
y_pred = gd.best_estimator_.predict(X_pred)

# output predictions dataframe
temp = pd.DataFrame(pd.read_csv("/content/drive/My Drive/UC Machine Learning/Datasets/test.csv")['PassengerId'])
temp['Survived'] = y_pred
temp.to_csv("/content/drive/My Drive/NLP/submission.csv", index = False)